In [3]:
from huggingface_hub import login
login()


In [4]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("kikiRa/Biblia-Portugues-Ticuna")

Generating train split:   0%|          | 0/15814 [00:00<?, ? examples/s]

In [5]:
from datasets import load_dataset
import pandas as pd

# Load the dataset
dataset = load_dataset("kikiRa/Biblia-Portugues-Ticuna")

# Assuming the dataset is in a format where 'train' is a valid split
# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Show the first few rows of the DataFrame
print(df.head())


   id                                        translation
0   0  {'pt': 'No princípio criou Deus o céu e a terr...
1   1  {'pt': 'E disse Deus: Haja luz; e houve luz.',...
2   2  {'pt': 'E viu Deus que era boa a luz; e fez De...
3   3  {'pt': 'E Deus chamou à luz Dia; e às trevas c...
4   4  {'pt': 'E disse Deus: Haja uma expansão no mei...


In [6]:
import pandas as pd

# Sample DataFrame (replace this with your actual DataFrame)
# data = {
#     'translation': [
#         {'pt': 'No princípio criou Deus o céu e a terra.'},
#         {'pt': 'E disse Deus: Haja luz; e houve luz.'},
#         {'pt': 'E viu Deus que era boa a luz; e fez Deus a separação entre a luz e as trevas.'},
#         {'pt': 'E Deus chamou à luz Dia; e às trevas chamou Noite.'},
#         {'pt': 'E disse Deus: Haja uma expansão no meio das águas; e haja separação entre águas e águas.'}
#     ]
# }

# Creating the DataFrame
temp1 = df

# Preprocess the 'translation' column
temp1['translation_text'] = temp1['translation'].apply(lambda x: x['pt'])

# Drop the original 'translation' column if not needed
temp1 = temp1.drop(columns=['translation'])

# Show the DataFrame
print(temp1)


         id                                   translation_text
0         0  No princípio criou Deus o céu e a terra. E a t...
1         1               E disse Deus: Haja luz; e houve luz.
2         2  E viu Deus que era boa a luz; e fez Deus separ...
3         3  E Deus chamou à luz Dia; e às trevas chamou No...
4         4  E disse Deus: Haja uma expansão no meio das ág...
...     ...                                                ...
15809  7727  E o Espírito e a esposa dizem: Vem. E quem ouv...
15810  7728  Porque eu testifico a todo aquele que ouvir as...
15811  7729  E, se alguém tirar quaisquer palavras do livro...
15812  7730  Aquele que testifica estas coisas diz: Certame...
15813  7731  A graça de nosso Senhor Jesus Cristo seja com ...

[15814 rows x 2 columns]


In [9]:
pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
from unidecode import unidecode

# Substitua o caminho pelo caminho real do arquivo CSV
file_path = '/kaggle/input/the-holy-quran/Portuguese.csv'

# Carregar o CSV com delimitador '|'
df = pd.read_csv(file_path, delimiter='|')

# Nome da coluna a ser renomeada
coluna_corrente = 'Em nome de Deus, o Clemente, o Misericordioso.'

# Verificar se a coluna existe
if coluna_corrente in df.columns:
    # Selecionar a coluna e renomear
    temp2 = df[[coluna_corrente]].copy()
    temp2.columns = ['translation_text']

    # Função para limpar o texto
    def clean_text(text):
        # Remover caracteres especiais e normalizar espaços
        text = unidecode(text)  # Remove acentuação e caracteres especiais
        text = text.replace('\x97', '-')  # Corrige caracteres especiais específicos, se necessário
        text = text.strip()  # Remove espaços em branco extras no início e no fim
        return text

    # Aplicar a função de limpeza ao DataFrame
    temp2['translation_text'] = temp2['translation_text'].apply(clean_text)

    # Exibir as primeiras linhas do DataFrame limpo
    print(temp2.head())
else:
    print(f"A coluna '{coluna_corrente}' não foi encontrada no DataFrame.")


                              translation_text
0       Louvado seja Deus, Senhor do Universo,
1                  Clemente, o Misericordioso,
2                     Soberano do Dia do Juzo.
3  S a Ti adoramos e s de Ti imploramos ajuda!
4                        Guia-nos  senda reta,


In [12]:
temp2.shape

(6235, 1)

In [13]:
# Supondo que temp1 já está definido e tem a mesma estrutura que temp2
# Exemplo: temp1 = pd.DataFrame({'translation_text': [...]})

# Concatenar temp1 e temp2 verticalmente
# Verifique se temp1 está definido
if 'temp1' in locals():
    combined_df = pd.concat([temp1, temp2], ignore_index=True)

    # Exibir as primeiras linhas do DataFrame combinado
    print(combined_df.head())
else:
    print("O DataFrame temp1 não está definido.")

    id                                   translation_text
0  0.0  No princípio criou Deus o céu e a terra. E a t...
1  1.0               E disse Deus: Haja luz; e houve luz.
2  2.0  E viu Deus que era boa a luz; e fez Deus separ...
3  3.0  E Deus chamou à luz Dia; e às trevas chamou No...
4  4.0  E disse Deus: Haja uma expansão no meio das ág...


In [14]:
combined_df.shape

(22049, 2)

In [17]:
pip install gensim pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.5 MB/s eta 0:00:00:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
import numpy as np
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import pandas as pd

class ModeloDeEmbeddingDeTexto:
    def __init__(self, tamanho_do_vetor=100, janela=5, contagem_minima=1):
        self.tamanho_do_vetor = tamanho_do_vetor
        self.janela = janela
        self.contagem_minima = contagem_minima
        self.tfidf_vectorizer = TfidfVectorizer()
        self.word2vec_modelo = None
        self.spell_checker = SpellChecker(language='pt')

    def preprocessar_texto(self, texto):
        """Preprocessa o texto para tokenização."""
        texto = texto.lower()
        return simple_preprocess(texto)

    def ajustar_tfidf(self, textos):
        """Ajusta o modelo TF-IDF aos textos fornecidos."""
        self.tfidf_vectorizer.fit(textos)

    def transformar_tfidf(self, textos):
        """Transforma os textos usando o modelo TF-IDF ajustado."""
        return self.tfidf_vectorizer.transform(textos)

    def treinar_word2vec(self, textos):
        """Treina o modelo Word2Vec com os textos fornecidos."""
        textos_tokenizados = [self.preprocessar_texto(texto) for texto in textos]
        self.word2vec_modelo = Word2Vec(
            sentences=textos_tokenizados,
            vector_size=self.tamanho_do_vetor,
            window=self.janela,
            min_count=self.contagem_minima,
            workers=4
        )

    def salvar_modelo_word2vec(self, caminho):
        """Salva o modelo Word2Vec em um arquivo especificado."""
        if self.word2vec_modelo:
            self.word2vec_modelo.save(caminho)
        else:
            raise ValueError("O modelo Word2Vec ainda não foi treinado.")

    def carregar_modelo_word2vec(self, caminho):
        """Carrega o modelo Word2Vec a partir de um arquivo especificado."""
        self.word2vec_modelo = Word2Vec.load(caminho)

    def obter_vetor_da_palavra(self, palavra):
        """Obtém o vetor de uma palavra específica do modelo Word2Vec."""
        if self.word2vec_modelo:
            return self.word2vec_modelo.wv[palavra]
        else:
            raise ValueError("O modelo Word2Vec ainda não foi treinado.")

    def gerar_texto(self, palavra_inicial, comprimento=10):
        """Gera um texto a partir de uma palavra inicial usando o modelo Word2Vec."""
        if not self.word2vec_modelo:
            raise ValueError("O modelo Word2Vec ainda não foi treinado.")
        
        palavra_atual = palavra_inicial
        texto_gerado = [palavra_atual]
        
        for _ in range(comprimento - 1):
            similares = self.word2vec_modelo.wv.most_similar(palavra_atual, topn=50)  # Aumenta o número de similares
            palavras_similares = [p for p, _ in similares]
            prob_similares = [prob for _, prob in similares]
            
            if not prob_similares:
                break
            
            # Amostragem ponderada com base na similaridade
            palavra_atual = np.random.choice(palavras_similares, p=prob_similares/np.sum(prob_similares))
            texto_gerado.append(palavra_atual)
        
        texto_gerado_corrigido = self.corrigir_ortografia(texto_gerado)
        return ' '.join(texto_gerado_corrigido)
    
    def corrigir_ortografia(self, palavras):
        """Corrige a ortografia das palavras usando o corretor ortográfico."""
        palavras_corrigidas = []
        for palavra in palavras:
            if palavra in self.spell_checker:
                palavras_corrigidas.append(palavra)
            else:
                sugestoes = self.spell_checker.candidates(palavra)
                if sugestoes:
                    # Usa o método `correction` para obter a melhor sugestão
                    palavra_corrigida = self.spell_checker.correction(palavra)
                    palavras_corrigidas.append(palavra_corrigida)
                else:
                    palavras_corrigidas.append(palavra)
        return palavras_corrigidas

    def contar_parametros(self):
        """Conta o número total de parâmetros do modelo Word2Vec."""
        if self.word2vec_modelo:
            num_palavras = len(self.word2vec_modelo.wv)
            tamanho_vetor = self.word2vec_modelo.vector_size
            total_parametros = num_palavras * tamanho_vetor
            return total_parametros
        else:
            raise ValueError("O modelo Word2Vec ainda não foi treinado.")

# Exemplo de uso
if __name__ == "__main__":
#     # Dados de exemplo
#     dados = {
#         'id': [0, 1, 2, 3, 4],
#         'translation_text': [
#             'No princípio criou Deus o céu e a terra.',
#             'E disse Deus: Haja luz; e houve luz.',
#             'E viu Deus que era boa a luz; e fez Deus a separação entre a luz e as trevas.',
#             'E Deus chamou à luz Dia; e às trevas chamou Noite.',
#             'E disse Deus: Haja uma expansão no meio das águas; e haja separação entre águas e águas.'
#         ]
#     }

#     df = pd.DataFrame(dados)
    textos = combined_df['translation_text']

    # Inicializa o modelo
    modelo = ModeloDeEmbeddingDeTexto()

    # Ajusta o TF-IDF
    modelo.ajustar_tfidf(textos)
    matriz_tfidf = modelo.transformar_tfidf(textos)
    print("Formato da Matriz TF-IDF:", matriz_tfidf.shape)

    # Treina o Word2Vec
    modelo.treinar_word2vec(textos)
    modelo.salvar_modelo_word2vec("word2vec.model")

    # Carrega o modelo Word2Vec e gera texto
    modelo.carregar_modelo_word2vec("word2vec.model")
    texto_gerado = modelo.gerar_texto('deus', comprimento=5)
    print("Texto gerado:", texto_gerado)

    # Conta e exibe a quantidade de parâmetros
    total_parametros = modelo.contar_parametros()
    print("Quantidade total de parâmetros:", total_parametros)


Formato da Matriz TF-IDF: (22049, 26622)
Texto gerado: deus agradável guardará desce daqui
Quantidade total de parâmetros: 2636700


In [19]:
modelo = ModeloDeEmbeddingDeTexto()
modelo.carregar_modelo_word2vec("word2vec.model")
texto_gerado = modelo.gerar_texto('amor', comprimento=10)
print("Texto gerado:", texto_gerado)

Texto gerado: amor exaltado louvado pura dizia fica sido cego leva excelente


In [20]:
modelo = ModeloDeEmbeddingDeTexto()
palavras = ['deus', 'amr', 'ortografia']
palavras_corrigidas = modelo.corrigir_ortografia(palavras)
print("Palavras corrigidas:", palavras_corrigidas)

Palavras corrigidas: ['deus', 'amor', 'ortografia']


In [21]:
modelo = ModeloDeEmbeddingDeTexto()
modelo.carregar_modelo_word2vec("word2vec.model")
vetor_deus = modelo.obter_vetor_da_palavra('deus')
vetor_amor = modelo.obter_vetor_da_palavra('amor')
print("Vetor de 'deus':", vetor_deus)
print("Vetor de 'amor':", vetor_amor)

Vetor de 'deus': [-0.16582318  0.07179627 -0.39120665 -0.07025646 -0.01296306 -0.5243184
  1.3023962   1.0032502  -0.04247101 -0.7879446   0.02592068 -0.5771454
 -0.73388284 -0.16542716 -0.1485097  -0.3580006  -0.986393   -0.82476705
 -0.91483235 -1.8988019   0.238753    0.12277885  0.77347517 -0.83789665
 -0.45329952 -1.1276271  -0.28043643 -1.0861782  -0.12710823 -0.4340333
  0.11110964  0.07232954  1.0826879  -0.65755725 -0.8529065   0.2268699
 -0.3276806  -1.0274975   0.28643858 -1.9206325   0.6537335  -0.66465163
  0.5368824  -0.5535221   0.439021   -0.9999717  -0.41331378 -1.0196916
  0.7141531   1.255208    0.05930662 -1.0332497   0.08693619 -0.35174146
  0.24859408  0.3850653   0.1887648  -0.17694518 -0.736922   -0.02985314
  0.07618988  0.07543006 -0.08357807 -0.7206633  -1.2894342   1.0152569
  0.872962    0.510468   -1.9548154   0.7453102   0.14699638  0.8977754
  0.831261    0.75710005 -0.09406514 -0.88594097 -0.3581071   0.2750632
 -0.49381548 -0.5503169  -0.14397717  0.19